Converting OHC code provided by John Fasullo to Python with xarray compatibility for use with Pangeo
by Julia Kent 
2019

In [18]:
#Import Modules
import numpy as np

Block 1 of IDL code:


New variables introduced: t, lev, noadj, fulldepth, m700, m300, m500, m5000, m2000, m100, lev_bnds, verbose, m1500, mthousand, dlim, t1

In [42]:
#Instead of using keywords as in IDL, or maping a dictionary that maps m100 to 100 meters, m300 to 300 meters, etc,
#We can set up an array of allowable depth values
#Why are only these depths allowed?
#Might be missing some functionality of the 5000 m case, how do lev and lev_bnds come into play?

def CheckIfDepthIsLegal(depth):
    DepthsInMeters = [100,300,500,700,1000,1500,2000, 5000]
    if depth in DepthsInMeters: print('True')
    else: print('Depths must be in meters. Allowable depths are', DepthsInMeters);

In [43]:
#Test 
CheckIfDepthIsLegal(100)
CheckIfDepthIsLegal(2)

True
Depths must be in meters. Allowable depths are [100, 300, 500, 700, 1000, 1500, 2000]


Block 2 of IDL Code:

New variables introduced: gd, minigood, levdim

In [45]:
#This code checks for allowable sea water temperature (268 K or -5 C)
#Not seeing how this is used yet, Code should check all temperatures in dataset.
def CheckIfTempIsLegal(temp):
    if temp >= 268: print('True')
    else: print('Temperatures must be in Kelvin. Minimum sea water temperature is 268 K.');

In [46]:
#Test 
CheckIfTempIsLegal(100)
CheckIfTempIsLegal(268)
CheckIfTempIsLegal(300)

Temperatures must be in Kelvin. Minimum sea water temperature is 268 K.
True
True
